<a href="https://colab.research.google.com/github/pragneshrana/SignalProcessing_MachineLearning/blob/main/LSTM_Human_Activity_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Signal Processing and Machine Learning
---



### What is Signal?
- Signal is just a fucntion form to represent the information
- It combines physical simulation, mathematics and information

### What is signal processing ?
- It convert signals into another form
- Information exraction and interpretation 
    - Speech recognition
    - Computer vision

#### Examples:
 - Audio Recording : analong pressure wave is converted into one-dimentional array
 - Photos : Analog scene of light is samples using CCD array and stored as a two-dimentional array
 - Text : Collection of char which further signes with 16 bit number
 - Ratings : movies, books are stores using 0-5
 

Process: 

Model : Voices -> Feature Extrcation -> Pitch / MMCC -> Model Training 

Unknown Voice : Feature Extraction -> Pitch / MMCC -> Prediction using Model -> Speaker


#### In case of human body, 
- Activity signal -> featre extraction -> Model training -> Activity Inference

(Ref : http://web.stanford.edu/class/ee269/Lecture1.pdf)

- Generate the dataset
    - Data sourcces
    - Simulation
    - Labelling and Supervised learning 
- Preprocess and tranform data
    - Pre-Processing
    - Transformation
    - Feature extraction
    - Visulization by time, frquency, and time-frequency domain
- Develope Predictive Models
    - Refrence Models
    - Hardware-Acceleated training 
    - Analyze and tune hyper-parameter
    - Time series models : Visual Inspection of over time
    - Frequency domain analysis: Time series domain shows signal's energy distributed over time whereas, frequency domain representation includes information on the phase shift that must be applied to each frequency component in order to recover the original time signal with a combination of all the individual frequency components (FFT o DFT)
    - **LSTM for human Activity recognition (HAR)**
        - sensor based HAR
            - wearables for humanbody
        - External Device
        - Sensory output data -> input data segmentation -> LSTM based model -> combine with per time step prediction 
        - Ref : https://www.mdpi.com/1424-8220/17/11/2556/htm
             - They have used three models, 
                 - Unidirectional LSTM
                 - Bidirectional LSTM
                 - Cascade Bidorectional LSTM
- Accelerate and Deploy
    - Apps
    - Enterprise Systems
    - Embedded Devices and Hardware
    
     

For the sake of example, Human Activity dataset is used and LSTM model is applied to predict human activity.


About this Dataset

The Human Activity Recognition database was built from the recordings of 30 study participants performing activities of daily living (ADL) while carrying a waist-mounted smartphone with embedded inertial sensors. The objective is to classify activities into one of the six activities performed.

In [3]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [4]:
train = pd.read_csv('https://raw.githubusercontent.com/pragneshrana/SignalProcessing_MachineLearning/main/data/train.csv')
test = pd.read_csv('https://raw.githubusercontent.com/pragneshrana/SignalProcessing_MachineLearning/main/data/test.csv')

In [5]:
train.columns


Index(['tBodyAcc-mean()-X', 'tBodyAcc-mean()-Y', 'tBodyAcc-mean()-Z',
       'tBodyAcc-std()-X', 'tBodyAcc-std()-Y', 'tBodyAcc-std()-Z',
       'tBodyAcc-mad()-X', 'tBodyAcc-mad()-Y', 'tBodyAcc-mad()-Z',
       'tBodyAcc-max()-X',
       ...
       'fBodyBodyGyroJerkMag-kurtosis()', 'angle(tBodyAccMean,gravity)',
       'angle(tBodyAccJerkMean),gravityMean)',
       'angle(tBodyGyroMean,gravityMean)',
       'angle(tBodyGyroJerkMean,gravityMean)', 'angle(X,gravityMean)',
       'angle(Y,gravityMean)', 'angle(Z,gravityMean)', 'subject', 'Activity'],
      dtype='object', length=563)

In [6]:
train.shape

(7352, 563)

In [7]:
train.head()


,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,tBodyAcc-max()-Y,tBodyAcc-max()-Z,tBodyAcc-min()-X,tBodyAcc-min()-Y,tBodyAcc-min()-Z,tBodyAcc-sma(),tBodyAcc-energy()-X,tBodyAcc-energy()-Y,tBodyAcc-energy()-Z,tBodyAcc-iqr()-X,tBodyAcc-iqr()-Y,tBodyAcc-iqr()-Z,tBodyAcc-entropy()-X,tBodyAcc-entropy()-Y,tBodyAcc-entropy()-Z,"tBodyAcc-arCoeff()-X,1","tBodyAcc-arCoeff()-X,2","tBodyAcc-arCoeff()-X,3","tBodyAcc-arCoeff()-X,4","tBodyAcc-arCoeff()-Y,1","tBodyAcc-arCoeff()-Y,2","tBodyAcc-arCoeff()-Y,3","tBodyAcc-arCoeff()-Y,4","tBodyAcc-arCoeff()-Z,1","tBodyAcc-arCoeff()-Z,2","tBodyAcc-arCoeff()-Z,3","tBodyAcc-arCoeff()-Z,4","tBodyAcc-correlation()-X,Y","tBodyAcc-correlation()-X,Z","tBodyAcc-correlation()-Y,Z",...,fBodyBodyAccJerkMag-entropy(),fBodyBodyAccJerkMag-maxInds,fBodyBodyAccJerkMag-meanFreq(),fBodyBodyAccJerkMag-skewness(),fBodyBodyAccJerkMag-kurtosis(),fBodyBodyGyroMag-mean(),fBodyBodyGyroMag-std(),fBodyBodyGyroMag-mad(),fBodyBodyGyroMag-max(),fBodyBodyGyroMag-min(),fBodyBodyGyroMag-sma(),fBodyBodyGyroMag-energy(),fBodyBodyGyroMag-iqr(),fBodyBodyGyroMag-entropy(),fBodyBodyGyroMag-maxInds,fBodyBodyGyroMag-meanFreq(),fBodyBodyGyroMag-skewness(),fBodyBodyGyroMag-kurtosis(),fBodyBodyGyroJerkMag-mean(),fBodyBodyGyroJerkMag-std(),fBodyBodyGyroJerkMag-mad(),fBodyBodyGyroJerkMag-max(),fBodyBodyGyroJerkMag-min(),fBodyBodyGyroJerkMag-sma(),fBodyBodyGyroJerkMag-energy(),fBodyBodyGyroJerkMag-iqr(),fBodyBodyGyroJerkMag-entropy(),fBodyBodyGyroJerkMag-maxInds,fBodyBodyGyroJerkMag-meanFreq(),fBodyBodyGyroJerkMag-skewness(),fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",subject,Activity
0,0.288585,-0.020294,-0.132905,-0.995279,-0.983111,-0.913526,-0.995112,-0.983185,-0.923527,-0.934724,-0.567378,-0.744413,0.852947,0.685845,0.814263,-0.965523,-0.999945,-0.999863,-0.994612,-0.994231,-0.987614,-0.943220,-0.407747,-0.679338,-0.602122,0.929294,-0.853011,0.359910,-0.058526,0.256892,-0.224848,0.264106,-0.095246,0.278851,-0.465085,0.491936,-0.190884,0.376314,0.435129,0.660790,...,-1.0,-0.936508,0.346989,-0.516080,-0.802760,-0.980135,-0.961309,-0.973653,-0.952264,-0.989498,-0.980135,-0.999240,-0.992656,-0.701291,-1.000000,-0.128989,0.586156,0.374605,-0.991990,-0.990697,-0.989941,-0.992448,-0.991048,-0.991990,-0.999937,-0.990458,-0.871306,-1.000000,-0.074323,-0.298676,-0.710304,-0.112754,0.030400,-0.464761,-0.018446,-0.841247,0.179941,-0.058627,1,STANDING
1,0.278419,-0.016411,-0.123520,-0.998245,-0.975300,-0.960322,-0.998807,-0.974914,-0.957686,-0.943068,-0.557851,-0.818409,0.849308,0.685845,0.822637,-0.981930,-0.999991,-0.999788,-0.998405,-0.999150,-0.977866,-0.948225,-0.714892,-0.500930,-0.570979,0.611627,-0.329549,0.284213,0.284595,0.115705,-0.090963,0.294310,-0.281211,0.085988,-0.022153,-0.016657,-0.220643,-0.013429,-0.072692,0.579382,...,-1.0,-0.841270,0.532061,-0.624871,-0.900160,-0.988296,-0.983322,-0.982659,-0.986321,-0.991829,-0.988296,-0.999811,-0.993979,-0.720683,-0.948718,-0.271958,-0.336310,-0.720015,-0.995854,-0.996399,-0.995442,-0.996866,-0.994440,-0.995854,-0.999981,-0.994544,-1.000000,-1.000000,0.158075,-0.595051,-0.861499,0.053477,-0.007435,-0.732626,0.703511,-0.844788,0.180289,-0.054317,1,STANDING
2,0.279653,-0.019467,-0.113462,-0.995380,-0.967187,-0.978944,-0.996520,-0.963668,-0.977469,-0.938692,-0.557851,-0.818409,0.843609,0.682401,0.839344,-0.983478,-0.999969,-0.999660,-0.999470,-0.997130,-0.964810,-0.974675,-0.592235,-0.485821,-0.570979,0.273025,-0.086309,0.337202,-0.164739,0.017150,-0.074507,0.342256,-0.332564,0.239281,-0.136204,0.173863,-0.299493,-0.124698,-0.181105,0.608900,...,-1.0,-0.904762,0.660795,-0.724697,-0.928539,-0.989255,-0.986028,-0.984274,-0.990979,-0.995703,-0.989255,-0.999854,-0.993238,-0.736521,-0

In [8]:
test.head()

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,tBodyAcc-max()-Y,tBodyAcc-max()-Z,tBodyAcc-min()-X,tBodyAcc-min()-Y,tBodyAcc-min()-Z,tBodyAcc-sma(),tBodyAcc-energy()-X,tBodyAcc-energy()-Y,tBodyAcc-energy()-Z,tBodyAcc-iqr()-X,tBodyAcc-iqr()-Y,tBodyAcc-iqr()-Z,tBodyAcc-entropy()-X,tBodyAcc-entropy()-Y,tBodyAcc-entropy()-Z,"tBodyAcc-arCoeff()-X,1","tBodyAcc-arCoeff()-X,2","tBodyAcc-arCoeff()-X,3","tBodyAcc-arCoeff()-X,4","tBodyAcc-arCoeff()-Y,1","tBodyAcc-arCoeff()-Y,2","tBodyAcc-arCoeff()-Y,3","tBodyAcc-arCoeff()-Y,4","tBodyAcc-arCoeff()-Z,1","tBodyAcc-arCoeff()-Z,2","tBodyAcc-arCoeff()-Z,3","tBodyAcc-arCoeff()-Z,4","tBodyAcc-correlation()-X,Y","tBodyAcc-correlation()-X,Z","tBodyAcc-correlation()-Y,Z",...,fBodyBodyAccJerkMag-entropy(),fBodyBodyAccJerkMag-maxInds,fBodyBodyAccJerkMag-meanFreq(),fBodyBodyAccJerkMag-skewness(),fBodyBodyAccJerkMag-kurtosis(),fBodyBodyGyroMag-mean(),fBodyBodyGyroMag-std(),fBodyBodyGyroMag-mad(),fBodyBodyGyroMag-max(),fBodyBodyGyroMag-min(),fBodyBodyGyroMag-sma(),fBodyBodyGyroMag-energy(),fBodyBodyGyroMag-iqr(),fBodyBodyGyroMag-entropy(),fBodyBodyGyroMag-maxInds,fBodyBodyGyroMag-meanFreq(),fBodyBodyGyroMag-skewness(),fBodyBodyGyroMag-kurtosis(),fBodyBodyGyroJerkMag-mean(),fBodyBodyGyroJerkMag-std(),fBodyBodyGyroJerkMag-mad(),fBodyBodyGyroJerkMag-max(),fBodyBodyGyroJerkMag-min(),fBodyBodyGyroJerkMag-sma(),fBodyBodyGyroJerkMag-energy(),fBodyBodyGyroJerkMag-iqr(),fBodyBodyGyroJerkMag-entropy(),fBodyBodyGyroJerkMag-maxInds,fBodyBodyGyroJerkMag-meanFreq(),fBodyBodyGyroJerkMag-skewness(),fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",subject,Activity
0,0.257178,-0.023285,-0.014654,-0.938404,-0.920091,-0.667683,-0.952501,-0.925249,-0.674302,-0.894088,-0.554577,-0.466223,0.717208,0.635502,0.789497,-0.877764,-0.997766,-0.998414,-0.934345,-0.975669,-0.949824,-0.830478,-0.168084,-0.378996,0.246217,0.521204,-0.487793,0.482280,-0.045462,0.211955,-0.134894,0.130858,-0.014176,-0.105971,0.073544,-0.171516,0.040063,0.076989,-0.490546,-0.709003,...,-0.484619,-1.000000,-0.035356,-0.254248,-0.700326,-0.770610,-0.797113,-0.764485,-0.820188,-0.937959,-0.770610,-0.970958,-0.798387,0.179435,-1.000000,-0.047391,-0.467849,-0.761326,-0.890165,-0.907308,-0.895301,-0.917883,-0.909829,-0.890165,-0.994105,-0.898022,-0.234815,-1.000000,0.071645,-0.330370,-0.705974,0.006462,0.162920,-0.825886,0.271151,-0.720009,0.276801,-0.057978,2,STANDING
1,0.286027,-0.013163,-0.119083,-0.975415,-0.967458,-0.944958,-0.986799,-0.968401,-0.945823,-0.894088,-0.554577,-0.806013,0.768031,0.683698,0.796706,-0.969097,-0.999580,-0.999646,-0.997720,-0.994006,-0.973637,-0.950720,-0.302437,-0.348243,-0.404785,0.507492,-0.156495,0.040674,0.272991,0.197566,-0.194556,0.411411,-0.340466,0.077555,-0.084024,0.035305,-0.010083,-0.104983,-0.429134,0.399177,...,-0.723642,-1.000000,-0.491213,-0.215404,-0.666004,-0.924461,-0.916774,-0.919383,-0.917983,-0.991121,-0.924461,-0.995727,-0.933127,-0.218860,-1.000000,-0.031474,-0.090655,-0.461801,-0.951977,-0.938212,-0.938008,-0.949922,-0.986902,-0.951977,-0.998272,-0.951255,-0.431053,-1.000000,-0.401189,-0.121845,-0.594944,-0.083495,0.017500,-0.434375,0.920593,-0.698091,0.281343,-0.083898,2,STANDING
2,0.275485,-0.026050,-0.118152,-0.993819,-0.969926,-0.962748,-0.994403,-0.970735,-0.963483,-0.939260,-0.568512,-0.799116,0.848305,0.667864,0.822442,-0.976782,-0.999954,-0.999570,-0.998720,-0.993456,-0.974266,-0.964976,-0.617916,-0.695490,-0.536693,0.242147,-0.114997,0.032703,0.192385,-0.011945,-0.063363,0.470819,-0.507395,0.188536,-0.231575,0.632120,-0.550708,0.305653,-0.323848,0.279786,...,-0.898000,-1.000000,0.140729,-0.206062,-0.639809,-0.975209,-0.973998,-0.967097,-0.982849,-0.997663,-0.975209,-0.999504,-0.9682

In [9]:
test.shape

(2947, 563)

In [10]:
X_train = train.iloc[:,:-2]
Y_train = train.iloc[:,-1]
print(X_train.shape)
print(Y_train.shape)

(7352, 561)
(7352,)


In [11]:
X_test = test.iloc[:,:-2]
Y_test = test.iloc[:,-1]
print(X_test.shape)
print(Y_test.shape)

(2947, 561)
(2947,)


In [42]:
TRAIN_LEN = len(X_train)  
TEST_LEN = len(X_test)  
NUM_STEPS = X_train.shape[0] 
NUM_FEATURES = X_train.shape[1]
NUM_HIDDEN = 32
NUM_CLASSES = 6
LR = 0.0025
LAMBDA = 0.0020
NUM_ITERS = TRAIN_LEN * 100  
BATCH_SIZE = 1024

print(X_test.shape, Y_test.shape, X_train.shape, Y_train.shape)
print(NUM_INPUT)

(2947, 561) (2947,) (7352, 561) (7352,)
561


In [13]:
def LSTM_RNN(_X, _weights, _biases):
    _X = tf.transpose(_X, [1, 0, 2])
    _X = tf.reshape(_X, [-1, NUM_INPUT]) 
    _X = tf.nn.relu(tf.matmul(_X, _weights['hidden']) + _biases['hidden'])
    _X = tf.split(_X, NUM_STEPS, 0) 
    lstm_cell_1 = tf.contrib.rnn.BasicLSTMCell(NUM_HIDDEN, forget_bias=1.0, state_is_tuple=True)
    lstm_cell_2 = tf.contrib.rnn.BasicLSTMCell(NUM_HIDDEN, forget_bias=1.0, state_is_tuple=True)
    lstm_cells = tf.contrib.rnn.MultiRNNCell([lstm_cell_1, lstm_cell_2], state_is_tuple=True)
    outputs, states = tf.contrib.rnn.static_rnn(lstm_cells, _X, dtype=tf.float32)
    lstm_last_output = outputs[-1]
    return tf.matmul(lstm_last_output, _weights['out']) + _biases['out']


def next_batch(_train, step, batch_size):
    shape = list(_train.shape)
    shape[0] = batch_size
    batch_s = np.empty(shape)

    for i in range(batch_size):
        index = ((step-1)*batch_size + i) % len(_train)
        batch_s[i] = _train[index] 

    return batch_s


def one_hot(y_):
    y_ = y_.reshape(len(y_))
    n_values = int(np.max(y_)) + 1
    return np.eye(n_values)[np.array(y_, dtype=np.int32)]

In [43]:

regressor = keras.Sequential([layers.LSTM(units = NUM_HIDDEN, activation = 'relu', return_sequences = True, input_shape = (NUM_STEPS, NUM_FEATURES)), #input shape looks like (batch_size, time_steps, units)
                        layers.LSTM(units =NUM_HIDDEN,  return_sequences = True),
                        layers.Dropout(0.2),
                        layers.LSTM(units=NUM_HIDDEN,unit_forget_bias=1.0),
                        layers.Dense(15),
                        layers.Dense(6)])
print(regressor.summary())



Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_55 (LSTM)               (None, 7352, 32)          76032     
_________________________________________________________________
lstm_56 (LSTM)               (None, 7352, 32)          8320      
_________________________________________________________________
dropout_21 (Dropout)         (None, 7352, 32)          0         
_________________________________________________________________
lstm_57 (LSTM)               (None, 32)                8320      
_________________________________________________________________
dense_36 (Dense)             (None, 15)                495       
_________________________________________________________________
dense_37 (Dense)             (None, 6)                 96        
Total params: 93,263
Trainable params: 93,263
Non-trainable params: 0
_________________________________________________

In [21]:
regressor.compile(optimizer='adam', loss = 'mean_squared_error')
regressor.fit(X_train, Y_train, epochs=DISP_ITER, batch_size=BATCH_SIZE)

correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

y_pred = regressor.predict(x)

Epoch 1/20000


ValueError: ignored

In [28]:
test_losses = []
test_accuracies = []
train_losses = []
train_accuracies = []


sess = tf.InteractiveSession(config=tf.ConfigProto(log_device_placement=True))
init = tf.global_variables_initializer()
sess.run(init)
saver = tf.train.Saver()

step = 1
while step * BATCH_SIZE <= NUM_ITERS:
    batch_xs = next_batch(X_Train, step, BATCH_SIZE)
    batch_ys = one_hot(next_batch(y_Train, step, BATCH_SIZE))

    
    _, loss, acc = sess.run([optimizer, cost, accuracy],feed_dict={x: batch_xs, y: batch_ys})
    train_losses.append(loss)
    train_accuracies.append(acc)
    
    
    if (step*BATCH_SIZE % DISP_ITER == 0) or (step == 1) or (step * BATCH_SIZE > NUM_ITERS):
        print("Iteration " + str(step*BATCH_SIZE) +  ":Batch Loss = " + "{:.3f}".format(loss) + ", Accuracy = {}".format(acc))
        loss, acc = sess.run([cost, accuracy], feed_dict={x: X_Test,y: one_hot(y_Test)})
        test_losses.append(loss)
        test_accuracies.append(acc)
        print("Test: " + "Batch Loss = {}".format(loss) + ", Accuracy = {}".format(acc))
    step += 1



one_hot_predictions, accuracy, final_loss = sess.run([pred, accuracy, cost],feed_dict={x: X_Test,y: one_hot(y_Test)})

test_losses.append(final_loss)
test_accuracies.append(accuracy)

print("RESULT: " + "Batch Loss = {}".format(final_loss) + ", Accuracy = {}".format(accuracy))



AttributeError: ignored

In [43]:
model = keras.Sequential([layers.Embedding(input_dim=1000, output_dim=64),
                          layers.LSTM(128),
                          layers.Dense(10)])
# Add an Embedding layer expecting input vocab of size 1000, and
# output embedding dimension of size 64.
# Add a LSTM layer with 128 internal units.
# Add a Dense layer with 10 units.
model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 64)          64000     
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               98816     
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1290      
Total params: 164,106
Trainable params: 164,106
Non-trainable params: 0
_________________________________________________________________


In [7]:
c = tf.constant([[1.0, 2.0,4.0,5.2], [3.0, 4.0,6,6.3],[3.0, 8.0,9.0,4.3],[8.0, 12.0,6.0,7.3]])
# d = tf.constant([[1.0, 1.0], [0.0, 1.0]])
# e = tf.matmul(c, d)
print(c)

tf.Tensor(
[[ 1.   2.   4.   5.2]
 [ 3.   4.   6.   6.3]
 [ 3.   8.   9.   4.3]
 [ 8.  12.   6.   7.3]], shape=(4, 4), dtype=float32)


In [9]:
tf.transpose(c,[1, 0, 2])

InvalidArgumentError: ignored